In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import missingno as msn
import datetime
from pandas_profiling import ProfileReport
import sklearn

In [2]:
df = pd.read_parquet("escooter_history_2022.parquet")

In [ ]:
df[(df.temp == 3526.0)]
#df.loc[df.temp == 3526.0, "temp"] = df.temp.div(100)

In [3]:
#df[(df.temp > 50)]
df.loc[df.temp > 50, "temp"] = df.temp.div(100)

In [3]:
# df = df.set_index('date')
df_slice1 = df.groupby(pd.Grouper(key="datetime",freq="D")).count()

In [ ]:
df_slice1

In [ ]:
px.scatter(df_slice1, x=df_slice1.index, y="registered_customer")

Vorgegebene Aufgaben

In [4]:
df_slice2 = df[df["registered_customer"] == False].copy()

maybe group nach stunde dann count und mean von temp und atemp
dann lineare regression

In [6]:
df_slice3 = df_slice2.groupby(pd.Grouper(key="datetime",freq="H")).agg({"registered_customer": "count","temp": "mean", "atemp": "mean", "windspeed": "mean", "humidity": "mean",}).reset_index()

In [7]:
df_slice3

,datetime,registered_customer,temp,atemp,windspeed,humidity
0,2020-01-04 00:00:00,3,9.84,14.395,0.0000,81.0
1,2020-01-04 01:00:00,8,9.02,13.635,0.0000,80.0
2,2020-01-04 02:00:00,5,9.02,13.635,0.0000,80.0
3,2020-01-04 03:00:00,3,9.84,14.395,0.0000,75.0
4,2020-01-04 04:00:00,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
17539,2022-01-03 19:00:00,9,10.66,12.880,11.0014,60.0
17540,2022-01-03 20:00:00,5,10.66,12.880,11.0014,60.0
17541,2022-01-03 21:00:00,4,10.66,12.880,11.0014,60.0
17542,2022-01-03 22:00:00,7,10.66,13.635,8.9981,56.0


In [8]:
df_slice3.dropna()

,datetime,registered_customer,temp,atemp,windspeed,humidity
0,2020-01-04 00:00:00,3,9.84,14.395,0.0000,81.0
1,2020-01-04 01:00:00,8,9.02,13.635,0.0000,80.0
2,2020-01-04 02:00:00,5,9.02,13.635,0.0000,80.0
3,2020-01-04 03:00:00,3,9.84,14.395,0.0000,75.0
6,2020-01-04 06:00:00,2,9.02,13.635,0.0000,80.0
...,...,...,...,...,...,...
17539,2022-01-03 19:00:00,9,10.66,12.880,11.0014,60.0
17540,2022-01-03 20:00:00,5,10.66,12.880,11.0014,60.0
17541,2022-01-03 21:00:00,4,10.66,12.880,11.0014,60.0
17542,2022-01-03 22:00:00,7,10.66,13.635,8.9981,56.0


In [9]:
df_slice3.corr()

,registered_customer,temp,atemp,windspeed,humidity
registered_customer,1.000000,0.459885,0.453973,0.091659,-0.356967
temp,0.459885,1.000000,0.987181,-0.027144,-0.073364
atemp,0.453973,0.987181,1.000000,-0.063259,-0.055496
windspeed,0.091659,-0.027144,-0.063259,1.000000,-0.291329
humidity,-0.356967,-0.073364,-0.055496,-0.291329,1.000000


In [13]:
px.imshow(df_slice3.corr(),text_auto=True,aspect="auto")

unterschied korrelationskoeffizient: temp r= 0,4598 und atemp r= 0,4539


2020-6-1 gecheckt
2021-1-4 gecheckt
2021-9-6 gecheckt

In [31]:
df_test = df_slice2[df_slice2.datetime.between("2020-9-7", "2020-9-8")]

In [38]:
df_test.loc[925453,"holiday"] = 1

In [42]:
df_test = df_slice2[df_slice2.datetime.between("2021-9-6", "2021-9-7")]

In [43]:
df_test[df_test.holiday == 0]

,datetime,holiday,workingday,weather,temp,atemp,humidity,windspeed,registered_customer


In [16]:
df_sclice4= df_slice2[(df_slice2.holiday==1)&(df_slice2.windspeed>35)].copy()

In [17]:
df_sclice4

,datetime,holiday,workingday,weather,temp,atemp,humidity,windspeed,registered_customer
929511,2020-09-07 22:04:56,1.0,0.0,"cloudy, mist",24.60,27.275,88.0,39.0007,False
929513,2020-09-07 22:05:52,1.0,0.0,"cloudy, mist",24.60,27.275,88.0,39.0007,False
929514,2020-09-07 22:06:09,1.0,0.0,"cloudy, mist",24.60,27.275,88.0,39.0007,False
929518,2020-09-07 22:07:14,1.0,0.0,"cloudy, mist",24.60,27.275,88.0,39.0007,False
929538,2020-09-07 22:21:15,1.0,0.0,"cloudy, mist",24.60,27.275,88.0,39.0007,False
...,...,...,...,...,...,...,...,...,...
1387221,2020-12-28 11:51:22,1.0,0.0,"clear, few clouds",13.94,14.395,46.0,39.0007,False
1387228,2020-12-28 11:53:35,1.0,0.0,"clear, few clouds",13.94,14.395,46.0,39.0007,False
1387233,2020-12-28 11:54:38,1.0,0.0,"clear, few clouds",13.94,14.395,46.0,39.0007,False
1387240,2020-12-28 11:57:49,1.0,0.0,"clear, few clouds",13.94,14.395,46.0,39.0007,False


In [14]:
df_sclice4.describe()

,holiday,workingday,temp,atemp,humidity,windspeed,registered_customer
count,207.0,207.0,207.000000,207.000000,207.000000,207.000000,207
unique,NaN,NaN,NaN,NaN,NaN,NaN,1
top,NaN,NaN,NaN,NaN,NaN,NaN,False
freq,NaN,NaN,NaN,NaN,NaN,NaN,207
mean,1.0,0.0,15.168019,16.834903,45.454106,37.095829,NaN
std,0.0,0.0,3.046673,4.048780,13.533725,1.471588,NaN
min,1.0,0.0,13.120000,13.635000,37.000000,35.000800,NaN
25%,1.0,0.0,13.940000,14.395000,37.000000,36.997400,NaN
50%,1.0,0.0,13.940000,14.395000,42.000000,36.997400,NaN
75%,1.0,0.0,15.580000,19.695000,46.000000,39.000700,NaN


In [18]:
df_slice5 = df_sclice4.groupby(pd.Grouper(key="datetime",freq="H")).agg({"registered_customer": "count","temp": "mean", "atemp": "mean", "windspeed": "mean", "humidity": "mean",})

In [19]:
df_slice5

,registered_customer,temp,atemp,windspeed,humidity
datetime,,,,,
2020-09-07 22:00:00,18,24.60,27.275,39.0007,88.0
2020-09-07 23:00:00,0,NaN,NaN,NaN,NaN
2020-09-08 00:00:00,0,NaN,NaN,NaN,NaN
2020-09-08 01:00:00,0,NaN,NaN,NaN,NaN
2020-09-08 02:00:00,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2020-12-28 07:00:00,0,NaN,NaN,NaN,NaN
2020-12-28 08:00:00,0,NaN,NaN,NaN,NaN
2020-12-28 09:00:00,0,NaN,NaN,NaN,NaN


In [20]:
df_slice5.dropna()

,registered_customer,temp,atemp,windspeed,humidity
datetime,,,,,
2020-09-07 22:00:00,18,24.60,27.275,39.0007,88.0
2020-11-13 10:00:00,38,13.12,13.635,36.9974,42.0
2020-11-13 13:00:00,57,15.58,19.695,36.9974,37.0
2020-12-28 10:00:00,51,13.94,14.395,35.0008,42.0
2020-12-28 11:00:00,43,13.94,14.395,39.0007,46.0


In [18]:
df_slice5.corr()

,registered_customer,temp,atemp,windspeed,humidity
registered_customer,1.000000,-0.775230,-0.623625,-0.626736,-0.915651
temp,-0.775230,1.000000,0.967912,0.525291,0.950670
atemp,-0.623625,0.967912,1.000000,0.485406,0.844940
windspeed,-0.626736,0.525291,0.485406,1.000000,0.585789
humidity,-0.915651,0.950670,0.844940,0.585789,1.000000


In [22]:
px.imshow(df_slice5.corr(),text_auto=True,aspect="auto")

In [ ]:
# X = df_slice3["temp"]
# Y = df_slice3["registered_customer"]
# X.array.reshape(-1,1)
#array.reshape(-1, 1)

In [ ]:
# from sklearn import datasets, linear_model
# from sklearn.metrics import mean_squared_error, r2_score
# regr = linear_model.LinearRegression()
# regr.fit(X,Y)
# print("inputs", X.columns,"\n")
# print("Corresponding: \n", regr.coef_)
# print("Intercept: \n", regr.intercept_)

In [ ]:
#profile = ProfileReport(df_slice2, title='Idk', vars={"num": {"low_categorical_threshold": 0}})
#profile.to_notebook_iframe()

Maybe gründe warum temperatur so ausschlägt maybe bestimmter tag im monat
Trendlineien für nächste Monate/Jahre/Jahreszeiten
Herausfinden warum ende des monats so oft ausgeliehen wird

